In [47]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [42]:
dfs = []
for name in os.listdir('./player_game_logs'):
    dfs.append(pd.read_csv(f"./player_game_logs/{name}/{name}_SUMMARY.csv"))
combined_df = pd.concat(dfs, ignore_index=True).set_index('model_type')

In [43]:
summary_df = combined_df.groupby(combined_df.index).mean()
pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_columns', None)
summary_df

,n_train,n_test,mse_train,rmse_train,r2_train,mse_test,rmse_test,r2_test,y_test_mean,bias,Game of season_weight,Game for player_weight,Age (days)_weight,Home?_weight,Game Started?_weight,Year_weight,Month of Season_weight,Team_GSW_weight,Opponent_GSW_weight,Team_DET_weight,Opponent_DET_weight,Team_WSB_weight,Opponent_WSB_weight,Team_LAC_weight,Opponent_LAC_weight,Team_DAL_weight,Opponent_DAL_weight,Team_BOS_weight,Opponent_BOS_weight,Team_MEM_weight,Opponent_MEM_weight,Team_UTA_weight,Opponent_UTA_weight,Team_MIN_weight,Opponent_MIN_weight,Team_NJN_weight,Opponent_NJN_weight,Team_TOR_weight,Opponent_TOR_weight,Team_BRK_weight,Opponent_BRK_weight,Team_DEN_weight,Opponent_DEN_weight,Team_WAS_weight,Opponent_WAS_weight,Team_PHI_weight,Opponent_PHI_weight,Team_POR_weight,Opponent_POR_weight,Team_IND_weight,Opponent_IND_weight,Team_MIL_weight,Opponent_MIL_weight,Team_SAC_weight,Opponent_SAC_weight,Team_ATL_weight,Opponent_ATL_weight,Team_ORL_weight,Opponent_ORL_weight,Team_VAN_weight,Opponent_VAN_weight,Team_OKC_weight,Opponent_OKC_weight,Team_LAL_weight,Opponent_LAL_weight,Team_SDC_weight,Opponent_SDC_weight,Team_NOH_weight,Opponent_NOH_weight,Team_NYK_weight,Opponent_NYK_weight,Team_CHH_weight,Opponent_CHH_weight,Team_SEA_weight,Opponent_SEA_weight,Team_NOP_weight,Opponent_NOP_weight,Team_NOK_weight,Opponent_NOK_weight,Team_PHO_weight,Opponent_PHO_weight,Team_CHA_weight,Opponent_CHA_weight,Team_MIA_weight,Opponent_MIA_weight,Team_CLE_weight,Opponent_CLE_weight,Team_CHO_weight,Opponent_CHO_weight,Team_CHI_weight,Opponent_CHI_weight,Team_SAS_weight,Opponent_SAS_weight,Team_KCK_weight,Opponent_KCK_weight,Team_HOU_weight,Opponent_HOU_weight
model_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Lasso,402.0171,101.0063,34.6428,5.7129,0.0477,35.1239,5.7259,0.0265,8.7449,8.3199,0.0000,0.0004,0.0409,0.0003,0.6546,0.0005,0.0000,0.0094,0.0000,0.0061,0.0000,0.0057,0.0000,0.0055,0.0000,0.0043,0.0000,0.0051,0.0000,-0.0040,0.0000,0.0049,0.0000,0.0048,0.0000,0.0085,0.0000,0.0046,0.0000,-0.0002,0.0000,0.0108,0.0000,0.0019,0.0000,0.0091,0.0000,0.0046,0.0000,0.0093,0.0000,0.0040,0.0000,0.0018,0.0000,0.0085,0.0000,0.0109,0.0000,0.0017,0.0000,-0.0029,0.0000,-0.0050,0.0000,0.0000,0.0000,0.0027,0.0000,0.0018,0.0000,0.0018,0.0000,0.0077,0.0000,0.0001,0.0000,0.0000,0.0000,0.0031,0.0000,0.0036,0.0000,0.0020,0.0000,-0.0003,0.0000,0.0033,0.0000,0.0015,0.0000,-0.0030,0.0000,0.0009,0.0000,0.0069,0.0000
LinReg,402.0171,101.0063,24.6829,4.7838,0.3372,"967,609,371,024,295,842,545,664.0000","109,882,059,977.8117","-28,802,860,475,967,061,622,784.0000",8.7449,"-148,976,774,418.1524","2,063,672,095,980.2119","-2,063,672,095,978.3381",0.1518,0.2561,"39,723,741,719.2936",-0.5979,-0.4375,"202,730,332,944.3262","165,212,290,079.8911","655,898,280,080.2975","112,099,261,004.2770","94,448,208,592.2380","-583,256,900,660.9479","250,633,413,064.3935","74,180,214,190.6227","234,840,666,402.5617","113,179,915,613.1286","-217,610,382,724.7328","86,363,639,452.6838","-103,229,946,563.7150","134,189,267,402.8597","-29,625,315,781.1379","83,270,275,106.9926","186,819,066,543.3286","247,791,335,614.4087","194,670,665,950.8740","72,539,308,905.3854","817,609,215,546.6450","154,300,069,066.2872","67,643,174,462.1048","25,901,800,055.4824","351,980,973,199.0768","153,818,705,368.9506","-46,283,336,652.2670","211,092,793,504.6075","-185,105,221,632.4011","82,234,964,476.5919","329,243,838,186.7130","131,557,898,497.0161","120,815,144,249.0456","135,802,040,207.2835","-99,321,780,787.7578","114,815,261,906.0329","265,872,510,847.4184","107,715,602,970.4321","236,943,529,912.1367","27,826,968,548.0326","307,953,364,069.8708","114,613,831,862.9159","-300,569,175,257.7212","40,510,300,496.7839","-404,002,933,914.0988","-40,837,903,324.0544","-417,439,794,463.2264","87,854,675,942.6641","52,945,934,323.0535","-25,816,977,161.4017","154,732,035,522.5981","114,933,523,503.1067","-161,885,085,899.5002","127,304,533,743.6776","144,796,855,

In [49]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

train = pd.read_csv(f"./player_game_logs/nikola_jokić/nikola_jokić_TRAIN.csv")
test = pd.read_csv(f"./player_game_logs/nikola_jokić/nikola_jokić_TEST.csv")

X_train = train.drop(columns=['PTS'])
X_test = test.drop(columns=['PTS'])
y_train = train['PTS']
y_test = test['PTS']
n_train = y_train.count()
n_test = y_test.count()
test_mean = np.mean(y_test)

# scale features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_train_pred = model.predict(X_train_scaled)
mse_train = mean_squared_error(y_train, y_train_pred)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(y_train, y_train_pred)

y_pred_test = model.predict(X_test_scaled)
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test, y_pred_test)
print(y_pred_test)
  

[10.83984375 23.45898438 21.98242188 20.34570312 11.43554688 20.3671875
 27.05859375 26.49804688 19.52929688 22.65820312 17.60351562 16.15625
 21.64648438 13.38085938 20.48242188 29.6015625  16.66601562 25.52929688
 13.75390625 10.80859375 16.6640625  16.57226562 18.48828125 14.25
 25.71484375 19.3984375  22.78125     7.32421875 16.20117188 12.08789062
 19.9609375  14.58398438 21.69726562 18.06640625 26.20703125 28.94140625
 14.43359375 26.60742188 22.8671875  14.34375    15.70898438 18.59375
 13.63085938 17.7109375  27.58007812 10.78125    17.3125     19.40820312
 23.9296875  25.73828125 28.28710938 29.09960938 27.125      15.75
  7.2734375  21.484375   28.19335938 23.65039062 15.25585938 13.5703125
 13.91992188 19.625      18.19921875 20.24609375 26.61132812 17.23828125
 28.40820312 14.5859375  30.2265625  30.38867188 11.41992188 17.2265625
 16.56835938 26.39453125 17.453125   17.6875     11.63085938 33.8046875
 11.96875    28.21484375 12.359375   27.4921875  26.98632812 19.12695312


In [52]:
print(r2_test)

0.04687908464017587


In [50]:
weights = {f"{col}_weight": coef for col, coef in zip(X_train.columns, model.coef_)}
bias = model.intercept_

print(weights)
print(bias)

{'Game of season_weight': 20.909212284096398, 'Game for player_weight': -23.753981374605466, 'Age (days)_weight': -4.078945682307728, 'Home?_weight': 1.050192387607125, 'Game Started?_weight': 3.43582744536089, 'Year_weight': 22.059303722328576, 'Month of Season_weight': 5.673490447212962, 'Team_GSW_weight': -4332197787991.6724, 'Opponent_GSW_weight': -12497418548842.117, 'Team_DET_weight': -1099458572387.2656, 'Opponent_DET_weight': -12497418548844.607, 'Team_WSB_weight': 63897114120.90438, 'Opponent_WSB_weight': -1420940007888.2644, 'Team_LAC_weight': -181472142778.45517, 'Opponent_LAC_weight': -12497418548839.096, 'Team_DAL_weight': -15174836354.669483, 'Opponent_DAL_weight': -12497418548843.094, 'Team_BOS_weight': 2850458281.320808, 'Opponent_BOS_weight': -12497418548840.447, 'Team_MEM_weight': -136188322.03454974, 'Opponent_MEM_weight': -12497418548842.355, 'Team_UTA_weight': -2182319.3007307076, 'Opponent_UTA_weight': -12497418548840.902, 'Team_MIN_weight': -208879.92051540175, '

In [51]:
nj_test = pd.read_csv('./player_game_logs/nikola_jokić/nikola_jokić_SUMMARY.csv')
nj_test


,model_type,n_train,n_test,mse_train,rmse_train,r2_train,mse_test,rmse_test,r2_test,y_test_mean,bias,Game of season_weight,Game for player_weight,Age (days)_weight,Home?_weight,Game Started?_weight,Year_weight,Month of Season_weight,Team_GSW_weight,Opponent_GSW_weight,Team_DET_weight,Opponent_DET_weight,Team_WSB_weight,Opponent_WSB_weight,Team_LAC_weight,Opponent_LAC_weight,Team_DAL_weight,Opponent_DAL_weight,Team_BOS_weight,Opponent_BOS_weight,Team_MEM_weight,Opponent_MEM_weight,Team_UTA_weight,Opponent_UTA_weight,Team_MIN_weight,Opponent_MIN_weight,Team_NJN_weight,Opponent_NJN_weight,Team_TOR_weight,Opponent_TOR_weight,Team_BRK_weight,Opponent_BRK_weight,Team_DEN_weight,Opponent_DEN_weight,Team_WAS_weight,Opponent_WAS_weight,Team_PHI_weight,Opponent_PHI_weight,Team_POR_weight,Opponent_POR_weight,Team_IND_weight,Opponent_IND_weight,Team_MIL_weight,Opponent_MIL_weight,Team_SAC_weight,Opponent_SAC_weight,Team_ATL_weight,Opponent_ATL_weight,Team_ORL_weight,Opponent_ORL_weight,Team_VAN_weight,Opponent_VAN_weight,Team_OKC_weight,Opponent_OKC_weight,Team_LAL_weight,Opponent_LAL_weight,Team_SDC_weight,Opponent_SDC_weight,Team_NOH_weight,Opponent_NOH_weight,Team_NYK_weight,Opponent_NYK_weight,Team_CHH_weight,Opponent_CHH_weight,Team_SEA_weight,Opponent_SEA_weight,Team_NOP_weight,Opponent_NOP_weight,Team_NOK_weight,Opponent_NOK_weight,Team_PHO_weight,Opponent_PHO_weight,Team_CHA_weight,Opponent_CHA_weight,Team_MIA_weight,Opponent_MIA_weight,Team_CLE_weight,Opponent_CLE_weight,Team_CHO_weight,Opponent_CHO_weight,Team_CHI_weight,Opponent_CHI_weight,Team_SAS_weight,Opponent_SAS_weight,Team_KCK_weight,Opponent_KCK_weight,Team_HOU_weight,Opponent_HOU_weight
0,LinReg,512,129,58.8310,7.6701,0.3720,82.8368,9.1015,0.0469,20.1860,"12,497,418,548,847.5078",20.9092,-23.7540,-4.0789,1.0502,3.4358,22.0593,5.6735,"-4,332,197,787,991.6719","-12,497,418,548,842.1152","-1,099,458,572,387.2656","-12,497,418,548,844.6074","63,897,114,120.9044","-1,420,940,007,888.2644","-181,472,142,778.4552","-12,497,418,548,839.0957","-15,174,836,354.6695","-12,497,418,548,843.0938","2,850,458,281.3208","-12,497,418,548,840.4473","-136,188,322.0345","-12,497,418,548,842.3555","-2,182,319.3007","-12,497,418,548,840.9023","-208,879.9205","-12,497,418,548,840.1504","4,213.4438","5,152.7226",-450.8984,"-12,497,418,548,839.6504",6.4750,"-12,497,418,548,840.3262",-0.0527,0.0000,0.0000,"-12,497,418,548,838.3594",0.0000,"-12,497,418,548,840.9082",0.0000,"-12,497,418,548,840.5234",0.0000,"-12,497,418,548,835.2520",0.0000,"-12,497,418,548,837.1406",0.0000,"-12,497,418,548,840.5234",0.0000,"-12,497,418,548,841.2246",0.0000,"-12,497,418,548,842.3711",0.0000,0.0000,0.0000,"-12,497,418,548,837.2031",0.0000,"-12,497,418,548,840.0859",0.0000,0.0000,0.0000,0.0000,0.0000,"-12,497,418,548,837.3887",0.0000,0.0000,0.0000,0.0000,0.0000,"-12,497,418,548,838.2480",0.0000,0.0000,0.0000,"-12,497,418,548,838.2695",0.0000,0.0000,0.0000,"-12,497,418,548,839.6914",0.0000,"-12,497,418,548,842.9844",0.0000,"-12,497,418,548,841.7871",0.0000,"-12,497,418,548,844.9219",0.0000,"-12,497,418,548,837.0703",0.0000,0.0000,0.0000,"-12,497,418,548,841.2285"
1,Lasso,512,129,78.8009,8.8770,0.1588,73.9699,8.6006,0.1489,20.1860,17.8896,0.0000,0.0000,5.8892,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,-0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,-0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,-0.0000,0.0000,-0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000
2,"Random Forest (100 estimators, 20 depth)",512,129,10.7012,3.2713,0.8858,72.6531,8.5237,0.1641,20.1860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [45]:
torrey_craig = pd.read_csv('./player_game_logs/torrey_craig/torrey_craig_SUMMARY.csv')
torrey_craig

,model_type,n_train,n_test,mse_train,rmse_train,r2_train,mse_test,rmse_test,r2_test,y_test_mean,bias,Game of season_weight,Game for player_weight,Age (days)_weight,Home?_weight,Game Started?_weight,Year_weight,Month of Season_weight,Team_GSW_weight,Opponent_GSW_weight,Team_DET_weight,Opponent_DET_weight,Team_WSB_weight,Opponent_WSB_weight,Team_LAC_weight,Opponent_LAC_weight,Team_DAL_weight,Opponent_DAL_weight,Team_BOS_weight,Opponent_BOS_weight,Team_MEM_weight,Opponent_MEM_weight,Team_UTA_weight,Opponent_UTA_weight,Team_MIN_weight,Opponent_MIN_weight,Team_NJN_weight,Opponent_NJN_weight,Team_TOR_weight,Opponent_TOR_weight,Team_BRK_weight,Opponent_BRK_weight,Team_DEN_weight,Opponent_DEN_weight,Team_WAS_weight,Opponent_WAS_weight,Team_PHI_weight,Opponent_PHI_weight,Team_POR_weight,Opponent_POR_weight,Team_IND_weight,Opponent_IND_weight,Team_MIL_weight,Opponent_MIL_weight,Team_SAC_weight,Opponent_SAC_weight,Team_ATL_weight,Opponent_ATL_weight,Team_ORL_weight,Opponent_ORL_weight,Team_VAN_weight,Opponent_VAN_weight,Team_OKC_weight,Opponent_OKC_weight,Team_LAL_weight,Opponent_LAL_weight,Team_SDC_weight,Opponent_SDC_weight,Team_NOH_weight,Opponent_NOH_weight,Team_NYK_weight,Opponent_NYK_weight,Team_CHH_weight,Opponent_CHH_weight,Team_SEA_weight,Opponent_SEA_weight,Team_NOP_weight,Opponent_NOP_weight,Team_NOK_weight,Opponent_NOK_weight,Team_PHO_weight,Opponent_PHO_weight,Team_CHA_weight,Opponent_CHA_weight,Team_MIA_weight,Opponent_MIA_weight,Team_CLE_weight,Opponent_CLE_weight,Team_CHO_weight,Opponent_CHO_weight,Team_CHI_weight,Opponent_CHI_weight,Team_SAS_weight,Opponent_SAS_weight,Team_KCK_weight,Opponent_KCK_weight,Team_HOU_weight,Opponent_HOU_weight
0,LinReg,324,82,19.5932,4.4264,0.1835,23.9308,4.8919,0.0950,5.8171,"52,919,927,795,364.6094",4.0792,-1.8184,3.8275,-0.1742,2.2182,-1.0760,-0.7623,"9,848,119,753,771.4648","-882,186,146,946.9913","-8,719,850,721,020.9707","-882,186,146,949.2587","4,062,036,848,703.7192","-2,767,350,900,049.5688","-8,646,831,312,510.4033","-882,186,146,948.6396","2,517,094,130,112.4658","-882,186,146,951.0277","-242,360,566,098.9901","-882,186,146,947.7841","2,553,650,225.8233","-882,186,146,947.6906","-68,078,394.3101","-882,186,146,949.7775","2,295,097.8273","-882,186,146,950.5310","-1,002,247.5122","-139,945.1847","15,330.9981","-882,186,146,948.3723",-438.9218,"-882,186,146,947.2175","-52,037,741,648,413.2031","-882,186,146,946.9590",0.0703,"-882,186,146,950.6973",0.0000,"-882,186,146,950.5903",0.0001,"-882,186,146,947.0391","-52,037,741,648,412.0312","-882,186,146,949.3789","-52,037,741,648,415.3906","-882,186,146,949.5977",0.0000,"-882,186,146,949.8750",0.0000,"-882,186,146,950.8682",0.0000,"-882,186,146,946.4453",0.0000,0.0000,0.0000,"-882,186,146,947.0059",0.0000,"-882,186,146,948.5957",0.0000,0.0000,0.0000,0.0000,0.0000,"-882,186,146,947.9961",0.0000,0.0000,0.0000,0.0000,0.0000,"-882,186,146,947.2461",0.0000,0.0000,"-52,037,741,648,413.1328","-882,186,146,949.4980",0.0000,0.0000,0.0000,"-882,186,146,947.5449",0.0000,"-882,186,146,950.3320",0.0000,"-882,186,146,948.1230","-52,037,741,648,413.8438","-882,186,146,949.1816",0.0000,"-882,186,146,946.6982",0.0000,0.0000,0.0000,"-882,186,146,947.7109"
1,Lasso,324,82,23.9976,4.8987,0.0000,26.5181,5.1496,-0.0029,5.8171,6.0926,0.0000,0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,0.0000,-0.0000,0.0000,0.0000,-0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0000,-0.0000,-0.0000,0.0000,-0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,"Random Forest (100 estimators, 20 depth)",324,82,3.5007,1.8710,0.8541,24.6504,4.9649,0.0678,5.8171,NaN,NaN,NaN,Na